In [1]:
import sys
sys.path.append("..")

from heritageconnector.utils.generic import paginate_list

import geocoder
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm

tqdm.pandas()

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def get_id_and_countryid(loc: str):
    g = geocoder.geonames(loc, key='heritageconnector')

    if g.error:
        raise Exception(f"Rate limit met. {g.error}")

    try:
        address = g.json.get('address', None)
        # found label must be in requested label
        # the API sometimes does weird things like 'sussex' -> 'eastbourne'
        assert address in loc
    except:
        # try disambiguating one level up e.g. "debden, essex, england" -> "essex, england"
        if "," in loc:
            loc = ", ".join(loc.split(", ")[1:])
            return get_id_and_countryid(loc)
        else:
            return {}
        
    return {"name": loc, "geonames_address":address, "geonames_id": g.geonames_id, "country_id": g.json.get('raw', {}).get('countryId', None)}

## 1. Create place names dataframe and save to disk

In [25]:
df = pd.read_pickle("../GITIGNORE_DATA/filtering_people_orgs_result.pkl")
df.columns

Index(['LINK_ID', 'PREFERRED_NAME', 'TITLE_NAME', 'FIRSTMID_NAME',
       'LASTSUFF_NAME', 'SUFFIX_NAME', 'HONORARY_SUFFIX', 'GENDER',
       'BRIEF_BIO', 'DESCRIPTION', 'NOTE', 'BIRTH_DATE', 'BIRTH_PLACE',
       'DEATH_DATE', 'DEATH_PLACE', 'CAUSE_OF_DEATH', 'NATIONALITY',
       'OCCUPATION', 'WEBSITE', 'AFFILIATION', 'LINGUISTIC_GROUP', 'TYPE',
       'REFERENCE_NUMBER', 'SOURCE', 'CREATE_DATE', 'UPDATE_DATE',
       'res_ALL_NOTES', 'res_WIKIDATA_IDs', 'res_URLS', 'qcodes_filtered'],
      dtype='object')

In [26]:
df_objects = pd.read_pickle("../GITIGNORE_DATA/objects_with_types.pkl")
df_objects.columns

Index(['MKEY', 'TITLE', 'ITEM_NAME', 'CATEGORY1', 'COLLECTOR',
       'PLACE_COLLECTED', 'DATE_COLLECTED', 'PLACE_MADE', 'CULTURE',
       'DATE_MADE', 'MATERIALS', 'MEASUREMENTS', 'EXTENT', 'DESCRIPTION',
       'ITEM_COUNT', 'PARENT_KEY', 'BROADER_TEXT', 'WHOLE_PART', 'ARRANGEMENT',
       'LANGUAGE_OF_MATERIAL', 'EDITION', 'OPTION1', 'OPTION2', 'OPTION3',
       'OPTION4', 'OPTION5', 'OPTION6', 'OPTION7', 'OPTION8', 'OPTION9',
       'OPTION10', 'OPTION11', 'OPTION12', 'OPTION13', 'OPTION14', 'OPTION15',
       'CREATE_DATE', 'UPDATE_DATE', 'ITEM_NAME_resolved'],
      dtype='object')

In [29]:
def process_place_name(loc):
    """Split by semicolon and return unique values"""
    if isinstance(loc, str):
        loc = loc.lower()
    
    if str(loc) == "nan":
        return loc
    else:
        split = loc.split("; ")
        return list(set(split))

df["BIRTH_PLACE_list"] = df["BIRTH_PLACE"].apply(process_place_name)
df["DEATH_PLACE_list"] = df["DEATH_PLACE"].apply(process_place_name)
df_objects["PLACE_MADE_list"] = df_objects["PLACE_MADE"].apply(process_place_name)
df_objects["PLACE_COLLECTED_list"] = df_objects["PLACE_COLLECTED"].apply(process_place_name)

In [30]:
places_counts = Counter(df.BIRTH_PLACE_list.dropna().sum() + df.DEATH_PLACE_list.dropna().sum() + df_objects.PLACE_MADE_list.dropna().sum() + df_objects.PLACE_COLLECTED_list.dropna().sum())
places_unique = list(places_counts.keys())
len(places_counts)

6392

In [39]:
places_df = pd.DataFrame(pd.Series(places_counts).sort_values(ascending=False) / sum(places_counts.values()) * 100).reset_index().rename(columns={'index': 'place name', 0: '% collection'})
places_df.head()

,place name,% collection
0,"london, greater london, england, united kingdom",16.353221
1,"england, united kingdom",8.322525
2,united kingdom,4.790470
3,france,4.418350
4,"manchester, manchester urban district, greater...",4.233834


In [10]:
percentage_cumulative = places_df["% collection"].cumsum()

def no_records_to_reach_percent(percent):
    return percentage_cumulative[percentage_cumulative > percent].index.min()

for p in [50, 75, 85, 95, 99]:
    n = no_records_to_reach_percent(p)
    print(f"{n} ({int(n/len(places_df)*100)}%) records required to reach {p}%")

11 (0%) records required to reach 50%
119 (1%) records required to reach 75%
392 (6%) records required to reach 85%
1724 (26%) records required to reach 95%
5096 (79%) records required to reach 99%


## 2. Load place names dataframe from disk and fill
We have to this in chunks as the geonames API has a rate limit of 1000 queries per hour.

In [11]:
places_df = pd.read_pickle("../GITIGNORE_DATA/places_disambiguation.pkl")
idx_paginated = paginate_list(places_df.index.tolist(), 300)

df_pages_list = [places_df.loc[page] for page in idx_paginated]

print(len(df_pages_list))
df_pages_list[0]

22


,place name,% collection
0,"london, greater london, england, united kingdom",16.353221
1,"england, united kingdom",8.322525
2,united kingdom,4.790470
3,france,4.418350
4,"manchester, manchester urban district, greater...",4.233834
...,...,...
295,"göttingen, braunschweig, lower saxony, germany",0.027793
296,"ealing, london, england, united kingdom",0.027021
297,"darmstadt, hesse, germany",0.027021
298,"bedford, bedfordshire, england, united kingdom",0.027021


In [12]:
PAGE_NO = 0

df_page = df_pages_list[PAGE_NO]

df_page["result"] = df_page["place name"].progress_apply(get_id_and_countryid)

Error the hourly limit of 1000 credits for heritageconnector has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for heritageconnector has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}


Exception: Rate limit met. the hourly limit of 1000 credits for heritageconnector has been exceeded. Please throttle your requests or use the commercial service.